## TWITTER 

In [ ]:
# FUENTE : 

In [ ]:
import os
import sys
import json

def tweets_n_edges(tweet_file):
    tweets=[]
    edges=[]

    for i in open(tweet_file,"r"):
        if i=="\n":
            next
        else:
            try:
                tweet = json.JSONDecoder().raw_decode(i)[0]
                usr_mentions= tweet['entities']['user_mentions']
                if len(usr_mentions)>0:
                    for ii in usr_mentions:
                        if tweet['user']['screen_name'] != ii['screen_name']:
                            edges.append((tweet['user']['screen_name'], ii['screen_name']))
                tweets.append(tweet)
            except: # if no user mentions, or something unexpected
                continue

    return (tweets,edges)

tweets,edges = tweets_n_edges("data_science_twitter.txt")

In [ ]:
print "There are %s tweets about data science this week, and %s user mentions!" % ( len(tweets), len(edges) )


In [ ]:
import networkx as nx

G=nx.DiGraph() # initiate a directed graph
G.add_edges_from(edges) # add edges to the graph from user mentions
ev_cent=nx.eigenvector_centrality(G,max_iter=10000) # compute eigenvector centrality

ev_tuple = []
for i in ev_cent.keys():
    ev_tuple.append((i,ev_cent[i]))
    
zip(range(1,11)[::-1],sorted(ev_tuple,key=lambda x: x[1])[-10:])[::-1] # get the top 10 network influencers

In [ ]:
import community

def get_communities(tweets, edges):
    G_un=nx.Graph()
    G_un.add_edges_from(edges)
    parts = community.best_partition(G_un)
    values = [parts.get(node) for node in G_un.nodes()]

    communities = {}

    for i in tweets:
        screen_name = i['user']['screen_name'].encode("ascii","ignore")
        raw_text = i['text'].encode("ascii","ignore")
        if screen_name in parts.keys() and i['lang'] in ('en','und'): # get english tweets
            comm_num = parts[screen_name]
            if comm_num in communities.keys():
                if screen_name in communities[comm_num].keys():
                    text = communities[comm_num][screen_name]['raw_text']
                    communities[comm_num][screen_name]['n_tweets'] += 1
                    communities[comm_num][screen_name]['raw_text'] = ' '.join([text, raw_text]) 
                else:
                    communities[comm_num][screen_name] = {
                        'raw_text' : raw_text,
                        'n_tweets' : 1 
                    }
            else:
                communities[comm_num] = {}
                communities[comm_num][screen_name] = {
                    'raw_text' : raw_text,
                    'n_tweets' : 1 
                }
        else:
            continue

    return communities

communities = get_communities(tweets,edges)

In [ ]:
community_size = []
for i in communities.keys():
    community_size.append((i,len(communities[i].keys())))

print "%s distinct communities were detected \n" % len(communities.keys())

print "Here are the top 10 most populous communities:\n"
for i,j in sorted(community_size,key=lambda x: x[1])[::-1][:10]:
    print "Community %s has %s members" % (i,j)

## INSTAGRAM HASHTAGS

In [1]:
import sys
import warnings
import json
import dateutil
import datetime
import numpy as np
import pandas as pd
from os import path
from datetime import timedelta

In [8]:
def get_tags(text):
    '''Return unique lists of hashtags from twitter text, by splitting text
    and returing unique strings that start with #'''
    tags = list( set(tag for tag in text.split() if tag.startswith('#')) )
    to_remove="$:;,.?!/\\*%@#"
    return map(lambda x: ''.join( c for c in x if  c not in to_remove ),tags)

tags = []
verbose=False
for i in open("hashtag_list01.csv", 'r').readlines():
    if i=="\n":
        next
    else:
        try:
            data = json.JSONDecoder().raw_decode(i)
            hash = list(data)[0]
            text = hash["text"].rstrip('\n\r') # remove newline characters
            clean_text = text.encode("ascii","ignore") # removes unicode
            date = dateutil.parser.parse(hash["created_at"])
            tags.append([map(lambda x:x.upper(),get_tags(clean_text)),date]) # make hashtags uppercase to avoid case-senstive duplications
        except:
            if verbose==True: print( "No useable data on this line:"), hash
            continue

tags.sort(key=lambda l_t: l_t[1]) # make sure the data are date sorted, otherwise searching will be not be fast

#tags_df = pd.DataFrame(tags)
#tags_df.columns = ['text', 'date']
multi_index = pd.MultiIndex(levels = [ ['text', 'date']], ['a', 'b']], labels = [[0, 0, 1, 1], [0, 1, 0, 1]])    
df = pd.DataFrame(columns=multi_index)


tags_df = tags_df.set_index('date')


def flatten(x):
    return [item for sublist in x for item in sublist]

tag_text = flatten(tags_df["text"].values.tolist())

TypeError: __new__() got an unexpected keyword argument 'labels'

ImportError: cannot import name 'imread' from 'scipy.misc' (/Applications/miniconda3/envs/espacios_ya/lib/python3.8/site-packages/scipy/misc/__init__.py)

In [4]:
def red_col(word, font_size, position, orientation, random_state=None, **kwargs):
    return random.choice(['hsl(49, 100%, 81%)', 'hsl(34, 99%, 65%)', 'hsl(8, 87%, 53%)'])

cloud_mask = imread("brain.png")
image_colors = ImageColorGenerator(cloud_mask)
wc = WordCloud(background_color="white", max_words=2000, mask=cloud_mask)
wordcloud = wc.generate(" ".join(tag_text))
plt.figure(figsize=(16,16))
plt.imshow(wc.recolor(color_func=red_col))
plt.axis("off")

ImportError: cannot import name 'imread' from 'scipy.misc' (/Applications/miniconda3/envs/espacios_ya/lib/python3.8/site-packages/scipy/misc/__init__.py)

In [9]:
total_word_count=[]
for i in set(tag_text):
    total_word_count.append((i,tag_text.count(i)))
    
number = 10
top_all = dict(sorted(total_word_count, key=lambda x: x[1])[-number:])

for i in top_all.keys():
    print ("The %s hashtag was seen about %d times" % (i,top_all[i]))

NameError: name 'tag_text' is not defined

In [ ]:
def get_time_groups(df, sec):
    '''Find groups of hashtags in 60 second windows.'''
    time_now = datetime.datetime.now()
    start = df.index.searchsorted(time_now - timedelta(seconds=sec))
    end = df.index.searchsorted(time_now)
    return df.ix[start:end]

top_rank = {}
for i in top_all.keys():
    top_rank[i] = []

start = tags_df.index.searchsorted(datetime.datetime.utcfromtimestamp(tags_df.ix[0:1].index.values.tolist()[0]/1e9))
end = 0

while end < len(tags_df.index):
    end = tags_df.index.searchsorted(datetime.datetime.utcfromtimestamp(
            tags_df.ix[start:start+1].index.values.tolist()[0]/1e9) + 
            timedelta(seconds=3600))
    
    new_df = tags_df.ix[start:end]
    hash_list = flatten(new_df["text"].values.tolist())

    hash_list_counts =  dict([(x,hash_list.count(x)) for x in set(hash_list)])
    raw_counts = [(x,hash_list_counts[x]) if x in hash_list_counts.keys() 
                   else (x,0) for x in top_rank.keys()]
    
    ranks = zip(range(1,len(raw_counts)+1),sorted(raw_counts,key=lambda x: x[1])[::-1])

    for i in ranks:
        top_rank[i[1][0]].append(i[0])

    start = end

In [ ]:
import plotly
from plotly.graph_objs import *#Scatter, Layout

plotly.offline.init_notebook_mode()

def data_plot(rank_dict):
    container = []

    for i in rank_dict.keys():
        ranks = rank_dict[i]
        container.append(Scatter(x=range(1,len(ranks)),y=ranks,name=i))
        
    layout= dict(
        xaxis = dict(title = 'Hour'),
        yaxis = dict(title = 'Relative Rank',autorange='reversed'),
        legend=dict(
          y=0.5,
          font=dict(size=12)))

    return (container,layout)

data,layout=data_plot(top_rank)
fig = Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [11]:
import networkx as nx
import itertools
import re
import datetime

from IPython import display
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

def blu_col(word, font_size, position, orientation, random_state=None, **kwargs):
    return random.choice(['hsl(107, 50%, 91%)', 'hsl(135, 44%, 76%)', 'hsl(198, 56%, 53%)','hsl(198, 56%, 53%)'])

def create_graph(list_of_lists):
    '''given a list conprising of lists of hashtags, construct a graph.
    If the list is greater than 2 or more, make sure to create all possible
    edges given the hashtag combination.  If the list is 0, do nothing and 
    if it is of size 1 then just create a node.'''
    
    G=nx.Graph()    
    for i in list_of_lists:
        if len(i)>1:
            G.add_edges_from(list(itertools.combinations(i,2)))
        elif len(i)==0:
            continue
        elif len(i)==1:
            G.add_node(*i)
    return G

class StdOutListener(StreamListener):
    def on_data(self, data):
        out = []
        verbose=False
        try:
            data = json.JSONDecoder().raw_decode(data)
            hash = list(data)[0]
            text = hash["text"].rstrip('\n\r') # remove newline characters
            clean_text = text.encode("ascii","ignore") # removes unicode
            date = datetime.datetime.now()
            out.append([map(lambda x:x.upper(),get_tags(clean_text)),date]) # make hashtags uppercase to avoid case-senstive duplications
        except:
            if verbose==True: print ("No useable data on this line:"), hash

        out.sort(key=lambda l_t: l_t[1]) # make sure the data are date sorted, otherwise searching will be not be fast

        if 's_t' not in globals():
            #global s_t
            s_t = pd.DataFrame(out)
            s_t.columns = ['text', 'date']
            s_t = s_t.set_index('date')
        else:
            #global s_t
            s_t_2 = pd.DataFrame(out)
            s_t_2.columns = ['text', 'date']
            s_t_2 = s_t_2.set_index('date')
            s_t = pd.concat([s_t,s_t_2])
        
        g =  create_graph(get_time_groups(s_t,120)["text"].values.tolist())
        to_prune = nx.isolates(g) # find isolate nodes
        g.remove_nodes_from(to_prune) # remove isolate nodes
        
        if nx.number_of_nodes(g) == 0: # if empty graph, keep listening
            return True
        try: # try to compute centrailty, if no convergence, move on
            ev_cent=nx.eigenvector_centrality(g,max_iter=10000) # compute eigenvector centrality
        except:
            return True
            
        ev_tuple = []
        for i in ev_cent.keys():
            ev_tuple.append((i.replace("#", ""),ev_cent[i]))
            
        global time
        
        if (datetime.datetime.now() - time) > timedelta(seconds=1): # allow refresh every ~x seconds, if tweets come in
            cloud_mask = imread("tweet.png")
            image_colors = ImageColorGenerator(cloud_mask)
            wc = WordCloud(background_color="white", max_words=2000, mask=cloud_mask)
            wordcloud = wc.generate_from_frequencies(ev_tuple)
            plt.figure(figsize=(12,12))
            plt.imshow(wc.recolor(color_func=blu_col))
            plt.axis("off")
            display.clear_output(wait=True)
            display.display(plt.gcf())
            time  = datetime.datetime.now()
            
        return True        

        def on_error(self, status):
            print (status)
            
consumer_key = '[your customer key here]'
consumer_secret = '[your consumer secret here]'
access_token = '[your access token here]'
access_token_secret = '[your access token secret here]'

if 's_t' in globals():
    del(s_t)

time = datetime.datetime.now()
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)
stream.filter(track=['data science', 'big data', 'machine learning'])